In [1]:
import json
import requests
from requests_toolbelt.multipart.encoder import MultipartEncoder
import subprocess
import os
import json
import requests
import csv
import pandas as pd
import time
from datetime import datetime

In [2]:
IP_PORT = "127.0.0.1:8000"
SERVER = "http://"+IP_PORT
MOBSF_BASE_PATH = "D:/PhD/Model_Building/Scanners_and_Decompilers/Scanners/MobSF"
APIKEY = "4a9fc3e94941f832d40975a251056079c9be5406891a934ded21a3c0ecdc5119"

In [3]:
APPLICATION_BASE_PATH = "D:/PhD/Model_Building/Stage_4_Building_Vulnerability_Dataset/4_Round_MobSF"

JSON_FILE_PATH = APPLICATION_BASE_PATH+"/Scanned_App_JSON_Files"
SOURCE_FILE = "Scanned_App_Details.csv"
SCAN_TYPE = "apk"
TYPE = "Scanned_App"

#JSON_FILE_PATH = APPLICATION_BASE_PATH+"/Scanned_Files_JSON_Files"
#SOURCE_FILE = "Scanned_Files_Details.csv"
#SCAN_TYPE = "studio" # for zip
#TYPE = "Scanned_Files"

#JSON_FILE_PATH = APPLICATION_BASE_PATH+"/Scanned_No_Source_JSON_Files"
#SOURCE_FILE = "Scanned_No_Source_Details.csv"
#SCAN_TYPE = "apk"
#TYPE = "Scanned_No_Source"

In [4]:
def start_MobSF_server():
    os.chdir(MOBSF_BASE_PATH)
    command = 'start run.bat '+IP_PORT
    subprocess.call(command, shell = True)
    time.sleep(30)

In [5]:
def get_source_code(app_name, app_hash, source_file):    
    
    headers = {'Authorization': APIKEY}
    data = {"hash": app_hash , "type":SCAN_TYPE, "file": source_file}
    response = requests.post(SERVER + '/api/v1/view_source', data=data, headers=headers)   

    json_object = json.loads(response.text)

    source_code = json_object["data"]        

    java_file = open(app_name+".java", "wb")
    java_file.write(source_code.encode("utf-8"))
    java_file.close()    
    source_code_df=pd.read_csv(app_name+".java", encoding='utf-8', header=None, delimiter="\n",skip_blank_lines=False).fillna("")
    os.remove(app_name+".java")
    
    return source_code_df

In [6]:
def generate_vulnerable_dataset():   
    
    scan_details = pd.read_csv(APPLICATION_BASE_PATH+"/"+SOURCE_FILE)       
           
    vulberability_dataset = []  
    
    for index, row in scan_details.iterrows():
        try:
            app_name = str(row['app_name'])
            app_hash = str(row['app_hash'])       

            print("Generating vulnerability Dataset from app : "+app_name)

            json_file = open(JSON_FILE_PATH+'/'+app_name+'.json')
            code_analysis_json_obj = json.load(json_file)["code_analysis"]


            json_file.close()        

            #Iterating through each issue
            for each_android_issue in code_analysis_json_obj:
                try:
                    for each_files in code_analysis_json_obj[each_android_issue]["files"]:                        
                        try:
                            filepath = str(each_files)
                            code_lines_str_list = str(code_analysis_json_obj[each_android_issue]["files"][each_files]).split(",")
                            code_lines = list(map(int, code_lines_str_list))

                            for i in range(len(code_lines)):
                                code_lines[i] = code_lines[i] - 1

                            all_code_lines = get_source_code(app_name, app_hash, filepath)

                            for index, row in all_code_lines.iterrows():
                                vulnerability_status = 0
                                if index in code_lines:
                                    vulnerability_status = 1

                                code = str(row[0]).strip()                    
                                attr_id = ""
                                attr_description = ""
                                attr_type = ""
                                attr_pattern = ""
                                attr_severity = ""
                                attr_input_case = ""
                                attr_cvss = ""
                                attr_cwe_id = ""
                                attr_cwe_desc = ""
                                attr_owasp_mobile = ""
                                attr_masvs = ""
                                attr_ref = ""

                                if vulnerability_status == 1:                        
                                    metadata = code_analysis_json_obj[each_android_issue]["metadata"]
                                    attr_id = str(metadata["id"])
                                    attr_description = str(metadata["description"])
                                    attr_type = str(metadata["type"])
                                    attr_pattern = str(metadata["pattern"])
                                    attr_severity = str(metadata["severity"])
                                    attr_input_case = str(metadata["input_case"])
                                    attr_cvss = str(metadata["cvss"])
                                    attr_cwe = str(metadata["cwe"])
                                    attr_cwe_split = attr_cwe.split(" ")
                                    attr_cwe_id = attr_cwe_split[0]
                                    attr_cwe_desc = attr_cwe[len(attr_cwe_id)+1:]
                                    attr_owasp_mobile = str(metadata["owasp-mobile"])
                                    attr_masvs = str(metadata["masvs"])
                                    try:
                                        attr_ref = str(metadata["ref"])
                                    except:
                                        attr_ref=""

                                csv_data_row = [attr_id, attr_description, attr_type, attr_pattern, code, attr_severity,
                                                    attr_input_case, attr_cvss, attr_cwe_id, attr_cwe_desc,
                                                    attr_owasp_mobile, attr_masvs, attr_ref, vulnerability_status]

                                vulberability_dataset.append(csv_data_row)

                        except Exception as ef:
                            print(str(ef)+" : in file : "+each_files)
                            continue
                            
                except Exception as ei:
                    print(str(ei)+" : for android issue : "+ each_android_issue)
                    continue
                       
        except Exception as ej:
            print(str(ej)+ " : for json file : "+app_name+".json")
            continue

    #data frame to csv                
    header = ['ID', 'Description', 'Type', 'Pattern', 'Code', 'Severity', 'Input Case', 'CVSS', 'CWE_ID', 'CWE_Desc', 'OWASP_Mobile', 'OWSAP_MASVS', 'Reference', 'Vulnerability_status']
    vulberability_dataframe = pd.DataFrame(vulberability_dataset, columns = header)
    
    vulberability_dataframe.drop_duplicates(subset=['Code', 'CWE_ID'], inplace=True)
    
    date_time_now = str(datetime.now()).replace('-', '').replace(' ', '').replace(':', '').replace('.', '')
    vulberability_dataframe.to_csv(APPLICATION_BASE_PATH+"/"+TYPE+'_Unprocessed_Dataset_'+date_time_now+'.csv', sep=',', encoding='utf-8', index=False)

In [7]:
def execute_process():
    print("********************Dataset Generation Started**************************")
    #start_MobSF_server()
    generate_vulnerable_dataset()
    print("********************Dataset Generation Completed**************************")
    

In [8]:
execute_process()

********************Dataset Generation Started**************************
Generating vulnerability Dataset from app : material-flashlight
Generating vulnerability Dataset from app : material-notes
Generating vulnerability Dataset from app : material-player
Generating vulnerability Dataset from app : material-tea-timer
Generating vulnerability Dataset from app : materialfbook
Generating vulnerability Dataset from app : materialistic
Generating vulnerability Dataset from app : materialos
Generating vulnerability Dataset from app : materialos-2
Generating vulnerability Dataset from app : materials-live-wallpaper
Generating vulnerability Dataset from app : mather
Generating vulnerability Dataset from app : mathtools
Generating vulnerability Dataset from app : matlog-libre
Generating vulnerability Dataset from app : matrix-calc
Generating vulnerability Dataset from app : matsuri
Generating vulnerability Dataset from app : mattermost-classic
Generating vulnerability Dataset from app : maximao

Generating vulnerability Dataset from app : mountie
Generating vulnerability Dataset from app : mounts2sd
Generating vulnerability Dataset from app : mouseapp
Generating vulnerability Dataset from app : move-certs
Generating vulnerability Dataset from app : movian-remote
Generating vulnerability Dataset from app : movie-db
Generating vulnerability Dataset from app : movim
Generating vulnerability Dataset from app : moving-polygons
Generating vulnerability Dataset from app : mozc-for-android
Generating vulnerability Dataset from app : mozillanlpbackend
Generating vulnerability Dataset from app : mpdroid
Generating vulnerability Dataset from app : mpv-remote
Generating vulnerability Dataset from app : mpvandroid
Generating vulnerability Dataset from app : mqttpublisher-plugin
Generating vulnerability Dataset from app : mrwhite
Generating vulnerability Dataset from app : msnake
Generating vulnerability Dataset from app : mta-fare-buster
Generating vulnerability Dataset from app : mtg-fami

Generating vulnerability Dataset from app : noprovider2push
Generating vulnerability Dataset from app : northdog-audio-compass
Generating vulnerability Dataset from app : nosurf-for-reddit
Generating vulnerability Dataset from app : notable-plus
Generating vulnerability Dataset from app : notally
Generating vulnerability Dataset from app : note-crypt-pro
Generating vulnerability Dataset from app : notecipher
Generating vulnerability Dataset from app : noteless
Generating vulnerability Dataset from app : notepad-2
Generating vulnerability Dataset from app : notepad-2-3
Generating vulnerability Dataset from app : notes
Generating vulnerability Dataset from app : notes-2
Generating vulnerability Dataset from app : notes-2-3
Generating vulnerability Dataset from app : notification-analyser
Generating vulnerability Dataset from app : notification-cron
Generating vulnerability Dataset from app : notification-dictionary
Generating vulnerability Dataset from app : notification-notes
Generating

Generating vulnerability Dataset from app : opensyncedlists
Generating vulnerability Dataset from app : opentimer
Generating vulnerability Dataset from app : opentopomap-viewer
Generating vulnerability Dataset from app : opentracks
Generating vulnerability Dataset from app : opentracks-osm-dashboard
Generating vulnerability Dataset from app : opentrail
Generating vulnerability Dataset from app : openvegemap
Generating vulnerability Dataset from app : openvpn-for-android
Generating vulnerability Dataset from app : openwebweaver
Generating vulnerability Dataset from app : openwnn
Generating vulnerability Dataset from app : openworkout
Generating vulnerability Dataset from app : openwort
Generating vulnerability Dataset from app : opmt
Generating vulnerability Dataset from app : opsu
Generating vulnerability Dataset from app : opus-1-music-player
Generating vulnerability Dataset from app : orbitallivewallpaper
Generating vulnerability Dataset from app : orgro
Generating vulnerability Data